In [107]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split # used for splitting training and testing data
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

torch.manual_seed(1)

In [108]:
plt.figure(figsize=(8,5))

# time steps per batch of data
seq_length = 100

time_steps = np.linspace(0, np.pi, seq_length + 1)

data = pd.read_csv('../Datasets/SectionData.csv')

x = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

<Figure size 576x360 with 0 Axes>

In [109]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, kernel_size, hidden_size, drop_prob):
        super(NeuralNetwork, self).__init__()
        self.model = nn.Sequential(
            nn.Conv1d(1, 1, kernel_size),
            #nn.Conv1d(1, 1, kernel_size),
            #nn.Relu()
            nn.Flatten(),
            nn.Dropout(drop_prob),
            nn.Linear(input_size - kernel_size + 1, hidden_size[0]),
            nn.Linear(hidden_size[0], hidden_size[1]),
            nn.Linear(hidden_size[1], hidden_size[2]),
            nn.LogSoftmax(dim=1), # NLLLoss
        )
        
    def forward(self, x):
        return self.model(x)

In [110]:
BEAT_TYPES_INDEX = {
    'N': 0,
    'L': 1,
    'R': 2,
    'A': 3,
    'V': 4,
    'F': 5,
}

def formatDataX(data):
    temp = torch.tensor(data).float()
    return temp.view(temp.size(0), 1, temp.size(1))

def formatDataY(data):
    return torch.tensor(data).float()

def preprocess(df):

    # drop ecgNum row
    df = df.drop(["ecgNum"], axis=1)
    # Classify the dependent and independent variables
    X = df.iloc[:, :-1].values
    labels = df.iloc[:, -1].values
    Y = np.array([np.insert(np.zeros(5), BEAT_TYPES_INDEX[label], 1) for label in labels])
    
    # split the data into train, validate, test
    X_temp, X_test, Y_temp, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
    X_train, X_val, Y_train, Y_val = train_test_split(X_temp, Y_temp, test_size=0.2, random_state=0)
    
    class_weights = compute_class_weight('balanced', ['N','L','R','A','V','F'], labels)
    
    print(class_weights)
    
    class_weights = class_weights * [10.0, 1, 1, 1, 1, 0.5]
    
    print(class_weights)
    
    return formatDataX(X_train), formatDataX(X_val), formatDataX(X_test), formatDataY(Y_train), formatDataY(Y_val), formatDataY(Y_test), formatDataY(class_weights)


In [111]:
#X_train, X_val, X_test, Y_train, Y_val, Y_test, weights = preprocess(data)
#print(X_train.size())
#print(X_val.size())
#print(X_test.size())
#print(Y_train.size())
#print(Y_val.size())
#print(Y_test.size())

In [112]:
# size of the input at each time step
# TODO: handle input sizes
input_size = 200
# kernel size
kernel_size = 20
# size of the hidden state and cell state at each time step
hidden_size = [64, 32, 6]
# dropout probability
drop_prob = 0.01

# instantiate the NN
neuralNet = NeuralNetwork(input_size, kernel_size, hidden_size, drop_prob)
print(neuralNet)

NeuralNetwork(
  (model): Sequential(
    (0): Conv1d(1, 1, kernel_size=(20,), stride=(1,))
    (1): Flatten()
    (2): Dropout(p=0.01, inplace=False)
    (3): Linear(in_features=181, out_features=64, bias=True)
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): Linear(in_features=32, out_features=6, bias=True)
    (6): LogSoftmax()
  )
)


In [113]:
# Mean Squared Error and Adam Optimizer with a learning rate of 0.01
# TODO: play with LR
#criterion = nn.MSELoss()
#optimizer = optim.SGD(net.parameters(), lr=0.01)
#optimizer = torch.optim.Adam(neuralNet.parameters(), lr=0.01)

In [114]:
epochs = 500
print_every = 45

X_train, X_val, X_test, Y_train, Y_val, Y_test, weights = preprocess(data)

[ 0.22417539  2.07594911  2.31045977  6.58444706  2.35330617 20.88632585]
[ 2.24175388  2.07594911  2.31045977  6.58444706  2.35330617 10.44316293]


In [115]:
def test(net, X, Y, print_conf=True):
    resValues, resIndices = torch.max(net(X), 1)
    length = len(resIndices.numpy())
    testValues, testIndices = torch.max(Y, 1)
    results = np.array([np.insert(np.zeros(5), index, 1) for index in resIndices])
    
    conf_matrix = confusion_matrix(resIndices.numpy(), testIndices.numpy())

    correct = 0
    for i in range(len(conf_matrix)):
        correct += conf_matrix[i,i]
        
    print("Accuracy: {:.2%}".format(correct/length))
    if print_conf:
        print("Confusion Matrix:")
        print(conf_matrix)
    print("\n")

In [116]:
# train the NN
def train(net, X, Y, num_epochs, weights):  
    criterion = nn.NLLLoss(weights) 
    
    #optimizer = optim.SGD(net.parameters(), lr=0.01)
    optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
    
    for epoch in range (num_epochs):
        optimizer.zero_grad()
        out = net(X)
        
        # NLLLoss
        actValues, actIndices = torch.max(Y, 1)
        loss = criterion(out, actIndices)
        
        loss.backward()
        optimizer.step()
        
        if (epoch+1) % 50 == 0:
            test(net, X_test, Y_test, False)
            print("Epoch {} of {} => Loss {:.4f}".format(epoch+1, num_epochs, loss.item()))
        else:
            print("Epoch {} of {} => Loss {:.4f}\r".format(epoch+1, num_epochs, loss.item()), end="")

    return net

In [117]:
trained_NN = train(neuralNet, X_train, Y_train, epochs, weights)

Accuracy: 74.85%=> Loss 1.1103


Accuracy: 74.85%=> Loss 1.0850




KeyboardInterrupt: 

In [106]:
test(trained_NN, X_train, Y_train)
test(trained_NN, X_test, Y_test)
test(trained_NN, X_val, Y_val)

Accuracy: 81.60%

Confusion Matrix:
[[46578  3503  2203   652  2704   299]
 [  474  1372    29    51   233     1]
 [  162    13  2286    39    94     2]
 [  208   119   142   840   205     5]
 [  209   130    12    52  1203     7]
 [   69    23     5     4   189   206]]
Accuracy: 81.99%

Confusion Matrix:
[[14683  1119   655   189   767    97]
 [  163   416     3    18    55     0]
 [   60     6   725    10    29     0]
 [   64    40    55   249    88     0]
 [   55    47     6    19   350     0]
 [   20     3     2     4    46    58]]
Accuracy: 82.01%

Confusion Matrix:
[[11696   859   495   157   667    69]
 [  113   352     6    28    62     1]
 [   42     2   584    12    27     0]
 [   52    22    37   202    56     1]
 [   49    40     1    15   299     1]
 [   25     3     4     3    44    55]]


In [74]:
torch.save(trained_NN, "../Models/TestingCNN.pt")

/Users/connorsparling/opt/anaconda3/envs/HealthMonitorNN/lib/python3.7/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type NeuralNetwork. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [ ]:
def evaluateNNData(data):
    x = torch.tensor(data).float()
    x = x.view(1, 1, x.size(1))
    resValues, resIndices = torch.max(trained_NN(x), 1)
    return resIndices[0]

def giveMeABad():
    testValues, testIndices = torch.max(Y_test, 1)
    for i in range(len(testIndices)):
        result = evaluateNNData(X_test[i])
        if result != 0:
            print(i)
            for x in X_test[i][0].numpy():
                print(str(x) + ",", end="")
            break

In [ ]:
giveMeABad()